# **Knowledge Graph Generation**


### Load data into Spark

In [1]:
import sys
sys.path.append("/usr/local/python-env/py39/lib/python3.9/site-packages")

import pyspark
print(pyspark.__version__)

print(sys.executable)

3.5.1
/bin/python3.9


### Initialze a SparkSession

Initialize a test session to ensure the SparkSession is working properly. This will connect to the resource manager node that is running the YARN cluster. If we visit the YARN web portal, we can see that the Spark application is running.

Ensuring the pyspark library is being accessed from my local usr directory.


In [2]:
import os
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.9'

In [3]:
import pkg_resources

sedona_version = pkg_resources.get_distribution("apache-sedona").version
print(f"Apache Sedona version: {sedona_version}")

Apache Sedona version: 1.5.1


In [4]:
print(os.environ['SPARK_HOME'])
print(os.environ['PYSPARK_PYTHON'])

/usr/local/spark/latest
/usr/bin/python3.9


In [5]:

from pyspark.sql import SparkSession
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.spark import *

spark = SparkSession.builder.master("spark://columbus-oh.cs.colostate.edu:30800").appName("MyApp").config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .config('spark.jars.packages',
            'org.apache.sedona:sedona-spark-3.5_2.12:1.5.1,'
            'org.datasyslab:geotools-wrapper:1.5.1-28.2') \
    .config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all') \
    .getOrCreate()

Skipping SedonaKepler import, verify if keplergl is installed


https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /s/chopin/a/grad/flarrieu/.ivy2/cache
The jars for the packages stored in: /s/chopin/a/grad/flarrieu/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ce567161-90a8-42ed-9e56-7bf7b65986bb;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-3.5_2.12;1.5.1 in central
	found org.apache.sedona#sedona-common;1.5.1 in central


:: loading settings :: url = jar:file:/usr/local/spark/3.5.0-with-hadoop3.3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.19.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in user-list
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.errorprone#error_prone_annotations;2.5.1 in central
	found org.apache.sedona#sedona-spark-common-3.5_2.12;1.5.1 in central


In [6]:
from sedona.register import SedonaRegistrator

SedonaRegistrator.registerAll(spark)

/tmp/ipykernel_652923/4046572303.py:3: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)
/s/chopin/a/grad/flarrieu/.local/lib/python3.9/site-packages/sedona/register/geo_registrator.py:45: DeprecationWarning: Call to deprecated function register (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  cls.register(spark)


True

## Now to make the app

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import split
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.functions import year  # used to extract year from date, could do this manually as well
from pyspark.sql import Window
from pyspark.sql.functions import sum as pyspark_sum


from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.spark import *
import geopandas as gpd

spark = SparkSession \
    .builder \
    .appName('GeoSpatialQueries_Freddy') \
    .master('spark://columbus-oh.cs.colostate.edu:30801') \
    .config("spark.yarn.resourcemanager.address", "columbia.cs.colostate.edu:30799") \
    .config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .config('spark.jars.packages',
            'org.apache.sedona:sedona-spark-3.5_2.12:1.5.1,'
            'org.datasyslab:geotools-wrapper:1.5.1-28.2') \
    .config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all') \
    .getOrCreate()

# Set log level to DEBUG
spark.sparkContext.setLogLevel("ERROR")

sedona = SedonaContext.create(spark)
SedonaRegistrator.registerAll(spark)

# create a logger
logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)
logger.info("Pyspark initialized...")

24/04/17 12:42:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
/tmp/ipykernel_652923/4152555376.py:35: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)
/s/chopin/a/grad/flarrieu/.local/lib/python3.9/site-packages/sedona/register/geo_registrator.py:45: DeprecationWarning: Call to deprecated function register (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  cls.register(spark)


## Load the datasets

Got the code for this from https://sedona.apache.org/1.5.1/tutorial/sql/

Load GeoJSON using Spark JSON Data Source:

Spark SQL's built-in JSON data source supports reading GeoJSON data. To ensure proper parsing of the geometry property, we can define a schema with the geometry property set to type 'string'. This prevents Spark from interpreting the property and allows us to use the ST_GeomFromGeoJSON function for accurate geometry parsing.

```python
schema = "type string, crs string, totalFeatures long, features array<struct<type string, geometry string, properties map<string, string>>>";
(sedona.read.json(geojson_path, schema=schema)
    .selectExpr("explode(features) as features") # Explode the envelope to get one feature per row.
    .select("features.*") # Unpack the features struct.
    .withColumn("geometry", f.expr("ST_GeomFromGeoJSON(geometry)")) # Convert the geometry string.
    .printSchema())
```

In [8]:
# Import the necessary module from py4j to interact with JVM
from py4j.java_gateway import java_import

# Import the Path class from Hadoop. This class is used to handle file paths in Hadoop.
java_import(spark._jvm, 'org.apache.hadoop.fs.Path')

# Define a function to recursively get all .json and .geojson files in a directory and its subdirectories
def get_files_recursive(path):
    # Use the listStatus method of the FileSystem class to get an array of FileStatus objects
    # Each FileStatus object represents a file or directory in the given path
    file_status_arr = fs.listStatus(spark._jvm.Path(path))
    
    # Initialize an empty list to hold the file paths
    file_paths = []
    
    # Loop through each FileStatus object in the array
    for file_status in file_status_arr:
        # If the FileStatus object represents a directory
        if file_status.isDirectory():
            # Call the get_files_recursive function with the directory path
            # This is a recursive call, which means the function calls itself
            # Add the returned file paths to the file_paths list
            file_paths += get_files_recursive(file_status.getPath().toString())
        # If the FileStatus object represents a file that ends with .json or .geojson
        elif file_status.getPath().getName().endswith(('.json', '.geojson')):
            # Add the file path to the file_paths list
            file_paths.append(file_status.getPath().toString())
    
    print(file_paths)
    # Return the list of file paths
    return file_paths

In [10]:

# Initialize a Hadoop file system 
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())

# Directory containing the files
json_directory = "hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/"

# Define the schema for the GeoJSON data
geojsonSchema = "type string, crs string, totalFeatures long, features array<struct<type string, geometry string, properties map<string, string>>>"


# Get a list of the JSON and GeoJSON files in the directory and its subdirectories
json_files = get_files_recursive(json_directory)

# Create a dictionary to hold the DataFrames
json_dataset_dataframes = {}

# Define the current and desired EPSG codes
current_epsg = "EPSG:3857"  # Web Mercator
desired_epsg = "EPSG:4326"  # WGS84

# Load each JSON file into a DataFrame and store it in the dictionary
for file_path in json_files:
    file_name = file_path.split('/')[-1]
    
    # Print the file path
    print(f"Processing file: {file_path}")
    
    # Read the GeoJSON file using the defined schema using sedona into a spark dataframe
    df = spark.read.schema(geojsonSchema).json(file_path, multiLine=True)
    
    # Explode the features array to create a row for each feature and select the columns
    df = (df
        .select(F.explode("features").alias("features"))
        .select("features.*")
        # Use Sedona's ST_GeomFromGeoJSON function to convert the geometry string to a geometry object
        .withColumn("geometry", F.expr("ST_GeomFromGeoJSON(geometry)"))
        # Transform the 'geometry' column from the current EPSG code to the desired EPSG code
        .withColumn("geometry", F.expr(f"ST_Transform(geometry, '{current_epsg}', '{desired_epsg}')"))
        )
    
    json_dataset_dataframes[file_name] = df

['hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/AgriculturalArea.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/Areas_diapiric_structures.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/Bathymetry.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/Calderas.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/CityBoundaries.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/CountryTerritories.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/Diapiric_trends.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/Diapirs.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/Dikes_sills.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/EcoRegions.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/Faults.geojson', 'hdfs://columbus-oh.cs.colostate.edu:30785/geospatial/input/Gas_fluid_seep.geo

## We can see that the datasets are not in a workable format

In [11]:
# Print the schemas

for key, value in json_dataset_dataframes.items():
    print(f"{key} Schema:")
    value.printSchema()

AgriculturalArea.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

Areas_diapiric_structures.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

Bathymetry.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

Calderas.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

CityBoundaries.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: ge